In [19]:
!pip install youtube-transcript-api -q

In [21]:
from youtube_transcript_api import YouTubeTranscriptApi
from agents import Agent, function_tool, Runner, ItemHelpers, RunContextWrapper
import re
import os
from openai.types.responses import ResponseTextDeltaEvent
from dotenv import load_dotenv
import asyncio

load_dotenv() # to load open ai secret key into my python environment

# Checking the access to the openai secret key
openai_api_key = os.getenv("OPENAI_API_KEY")
print("OpenAI Key:", openai_api_key[:8] + "..." if openai_api_key else "Key not found")

OpenAI Key: sk-1G1d2...


#### **Openai definition of an AI agent:**
- Agents are the core building block in your apps. 
- An agent is a **Large Language Model (LLM)**, configured with instructions and **tools**.

In [37]:
# define instructions
instructions = "You provide help with tasks related to YouTube Videos."

by adding `@function_tool` decorator you can easily convert any python function to a tool in OpenAI Agent SDK... Super cool!

In [42]:
@function_tool
def fetch_youtube_transcript(url: str) -> str:
    """
    Extract transcript with timestamps from a YouTube Video URL and format it for LLM consumption.

    Args:
        url (str): YouTube Video URL link

    Returns:
        Formatted transcript with timestamps, where each entry is on a enw line in the format: "[MM:SS] Text"
    """
    # Extract video ID from URL
    video_id_pattern = r'(?:v=|\/)([0-9A-Za-z_-]{11}).*'

    video_id_match = re.search(video_id_pattern, url)

    if not video_id_match:
        raise ValueError("Invalid YouTube URL")
    
    video_id = video_id_match.group(1)

    try: 
        transcript = YouTubeTranscriptApi.get_transcript(video_id)

        # format each entry with timestamp and text
        formatted_entries = []
        for entry in transcript:
            # convert seconds to MM:SS format 
            minutes = int(entry['start']//60)
            seconds = int(entry['start']%60)
            timestamp = f"[{minutes:02d}:{seconds:02d}]"
            
            formatted_entry = f"{timestamp} {entry['text']}"
            formatted_entries.append(formatted_entry)
        
        # join all entries with newlines
        return "\n".join(formatted_entries)

    except Exception as e:
        raise Exception(f"Error fetching transcript: {str(e)}")
    
# define agent
agent = Agent(
    name="YouTube Transcript Agent",
    instructions=instructions,
    tools=[fetch_youtube_transcript],
)

# define main chat function 
async def main():
    input_items = []

    print("===YouTube Transcript Agent===")
    print("Type 'exit' to end the conversation")
    print("Ask me anything about YouTube Videos!")

    while True: 
        # Get user input:
        user_input = input("\nYou: ").strip()
        input_items.append({
                "content": user_input,
                "role": "user"
                })
        
        # check for exit command:
        if user_input.lower() in ["exit", "quit", "bye"]:
            print("\nGoodbye!")
            break
        
        if not user_input:
            continue

        print("\nAgent: ", end="", flush=True)
        result = Runner.run_streamed(
            starting_agent=agent,
            input=input_items
        )
        async for event in result.stream_events():
            # We'll ignore the raw responses event deltas
            if event.type == "raw_response_event" and isinstance(event.data, ResponseTextDeltaEvent):
                print(event.data.delta, end="", flush=True)
            
            elif event.type == "agent_updated_stream_event": 
                continue

            elif event.type == "run_item_stream_event":    
                if event.item.type == "tool_call_item":
                    print("\n-- Fetching transcript...")
                
                elif event.item.type == "tool_call_output_item":
                    input_items.append({"content": f"Transcript:\n{event.item.output}", "role": "system"})
                    print("-- Transcript fetched.")

                elif event.item.type == "message_output_item":
                    input_items.append({
                        "content": f"{event.item.raw_item}", 
                        "role": "assistant"
                        })
                
                else:
                    pass  # Ignore other event types

        print("\n")  # Add a newline after each response

if __name__ == "__main__":
    await main()

===YouTube Transcript Agent===
Type 'exit' to end the conversation
Ask me anything about YouTube Videos!

Agent: 
-- Fetching transcript...
-- Transcript fetched.
Based on the debate transcript between former President Donald Trump and Vice President Kamala Harris, here are the main viewpoints and statements from each:

### Donald Trump:
- **Economy:** Trump focused on condemning Harris's economic plans, claiming they would worsen the deficit and increase inflation. He argued for cutting taxes substantially and claimed that his past economic policies led to significant growth and low inflation.
- **Immigration:** He criticized Harris for increased illegal immigration and crime, suggesting that it was damaging the country. He proposed using local police and the National Guard for mass deportations.
- **Abortion:** Trump emphasized his role in overturning Roe v. Wade, framing it as a win for democracy by returning the power to the states. He distanced himself from the idea of a national 